True probability p: 0.9779
Heads: 984 Tails: 16


alt.LayerChart(...)

# Monte Carlo Simulation with Metropolis-Hastings

TODO: Make it less jargon-y

This notebook demonstrates a simple Monte Carlo simulation using the Metropolis-Hastings algorithm.
We sample from a standard normal distribution (target distribution) even if we start far away from the true mean.
The notebook illustrates:


- The implementation of the Metropolis-Hastings algorithm
- The convergence of the running mean to the true mean (0)
- The evolution of the Markov chain over iterations


In [ ]:
# --- Imports ---
import numpy as np
import pandas as pd
import altair as alt

# For reproducibility, if you want to test anything
# np.random.seed(42)

# Define this for the demonstration. TODO; Maybe add a slider for the user to choose.
true_p = np.random.uniform(0, 1)  # e.g., 0.7 or random

# Set number of iterations
N = 1000

# Define random variable and compute heads and tails
coin_flips = np.random.binomial(1, true_p, size=N)
num_heads = np.sum(coin_flips)
num_tails = N - num_heads

print(f"True probability p: {true_p:.4f}")
print("Heads:", num_heads, "Tails:", num_tails)

# Define target distribution:
# Posterior alpha p^(num_heads) * (1 - p)^(num_tails), uniform prior
def target(p):
    if p <= 0 or p >= 1:
        return 0
    return (p ** num_heads) * ((1 - p) ** num_tails)

# APply Metropolis-Hastings algorithm.
# TODO: Turn into function.
num_iterations = 3000
proposal_std = 0.5  # Increase to allow broader exploration
chain = []

current_p = 0.5  # Starting guess
for i in range(num_iterations):
    proposed_p = current_p + np.random.normal(0, proposal_std)
    # Ensure candidate remains in [0, 1]
    proposed_p = np.clip(proposed_p, 0, 1)

    current_target = target(current_p)
    proposed_target = target(proposed_p)

    # If current_target == 0, accept if proposed_target > 0, else 0
    if current_target == 0:
        acceptance_prob = 1 if proposed_target > 0 else 0
    else:
        acceptance_prob = min(1, proposed_target / current_target)

    if np.random.rand() < acceptance_prob:
        current_p = proposed_p

    chain.append(current_p)

chain = np.array(chain)

# Compute running mean at each step
# TODO: Visualize this as a stochastic process, i.e. an animation of some kind.
running_mean = np.cumsum(chain) / (np.arange(num_iterations) + 1)

# Prepare DataFrame
df = pd.DataFrame({
    'Iteration': np.arange(num_iterations),
    'Chain Value': chain,
    'Running Mean': running_mean
})

# TODO: Turn into function as well
# Altair visualization of the data.
# Gray line for raw chain, including rejections
base = alt.Chart(df).encode(
    x=alt.X('Iteration:Q', title='Iteration')
)

chain_line = base.mark_line(
    color='lightgray',
    strokeWidth=1
).encode(
    y=alt.Y('Chain Value:Q', 
            title='Parameter Value',
            scale=alt.Scale(domain=[0, 1]))  # Force y-axis [0,1]
)

# Blue line to track running mean
running_mean_line = base.mark_line(
    color='blue',
    strokeWidth=2
).encode(
    y='Running Mean:Q'
)

# Red horizontal rule for the true probability of the coin
true_rule = alt.Chart(pd.DataFrame({'y': [true_p]})).mark_rule(
    color='red',
    strokeWidth=2
).encode(
    y='y:Q'
)

# Combine all the layers together
final_chart = alt.layer(
    chain_line,
    running_mean_line,
    true_rule
).properties(
    width=600,
    height=400,
    title=f'Convergence of Estimated p (True p = {true_p:.4f})'
)

final_chart.display()


True probability p: 0.1323
Heads: 125 Tails: 875


alt.LayerChart(...)